In [14]:
!git clone https://github.com/cph-cachet/LocalLeadAttention.git
%cd LocalLeadAttention


Cloning into 'LocalLeadAttention'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (29/29), done.
remote: Total 31 (delta 5), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (31/31), 7.41 MiB | 4.38 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/LocalLeadAttention/LocalLeadAttention


In [15]:
!pip install numpy scipy scikit-learn joblib pandas tabulate tqdm scikit-multilearn


In [16]:
def load_challenge_data(filename):
    # Example: Loading ECG signals and metadata
    with open(filename, 'r') as f:
        # Parse data and return signals, labels, etc.
        data = f.read()
    return data


In [17]:
import os
import requests
from concurrent.futures import ThreadPoolExecutor

# ✅ Base URL for the MIT-BIH Arrhythmia Database
BASE_URL = "https://physionet.org/files/mitdb/1.0.0/"

# ✅ Directory to save downloaded files
SAVE_DIR = "./mitdb/records/"
os.makedirs(SAVE_DIR, exist_ok=True)

# ✅ URL of the RECORDS file (list of all record names)
records_file_url = BASE_URL + "RECORDS"
records_file_path = os.path.join(SAVE_DIR, "RECORDS")

# ✅ Download the RECORDS file if it doesn't already exist
if not os.path.exists(records_file_path):
    print("📥 Downloading RECORDS file...")
    response = requests.get(records_file_url)
    response.raise_for_status()
    with open(records_file_path, "w") as f:
        f.write(response.text)
    print("✅ RECORDS file downloaded!")

# ✅ Read the list of valid records from the RECORDS file
with open(records_file_path, "r") as f:
    valid_records = [line.strip() for line in f.readlines()]

# ✅ Choose the first 20 records
selected_records = valid_records[:100]  # First 20 records

# ✅ Function to download .dat, .hea, and .atr files for a record
def download_record(record_name):
    """Download .dat, .hea, and .atr files for the given record."""
    for ext in [".dat", ".hea", ".atr"]:
        file_url = f"{BASE_URL}{record_name}{ext}"
        file_path = os.path.join(SAVE_DIR, f"{record_name}{ext}")

        # Skip download if the file already exists
        if os.path.exists(file_path):
            print(f"✅ Already downloaded: {record_name}{ext}")
            continue

        try:
            response = requests.get(file_url, stream=True)
            response.raise_for_status()
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):
                    file.write(chunk)
            print(f"✅ Successfully downloaded: {record_name}{ext}")
        except requests.exceptions.RequestException as e:
            print(f"❌ Failed to download {record_name}{ext}: {e}")

# ✅ Use multi-threading to download selected records
with ThreadPoolExecutor(max_workers=5) as executor:
    executor.map(download_record, selected_records)

print("✅ Download process completed!")


📥 Downloading RECORDS file...
✅ RECORDS file downloaded!
✅ Successfully downloaded: 102.dat
✅ Successfully downloaded: 103.dat
✅ Successfully downloaded: 101.dat
✅ Successfully downloaded: 100.dat
✅ Successfully downloaded: 102.hea
✅ Successfully downloaded: 104.dat
✅ Successfully downloaded: 103.hea
✅ Successfully downloaded: 100.hea
✅ Successfully downloaded: 101.hea
✅ Successfully downloaded: 104.hea
✅ Successfully downloaded: 102.atr
✅ Successfully downloaded: 103.atr
✅ Successfully downloaded: 100.atr
✅ Successfully downloaded: 101.atr
✅ Successfully downloaded: 104.atr
✅ Successfully downloaded: 105.dat
✅ Successfully downloaded: 107.dat
✅ Successfully downloaded: 106.dat
✅ Successfully downloaded: 108.dat
✅ Successfully downloaded: 105.hea
✅ Successfully downloaded: 109.dat
✅ Successfully downloaded: 107.hea
✅ Successfully downloaded: 106.hea
✅ Successfully downloaded: 108.hea
✅ Successfully downloaded: 105.atr
✅ Successfully downloaded: 109.hea
✅ Successfully downloaded: 107.at

In [18]:
!pip install wfdb


In [19]:
import wfdb

# Example: Load a record
record = wfdb.rdrecord("./mitdb/records/100")
signal = record.p_signal  # ECG signal
print("Signal shape:", signal.shape)  # Typically [N_samples, N_leads]


Signal shape: (650000, 2)


In [20]:
from scipy.signal import resample

def resample_signal(signal, original_rate=360, target_rate=500):
    """Resample the signal to the desired rate."""
    num_samples = int(signal.shape[0] * target_rate / original_rate)
    resampled_signal = resample(signal, num_samples, axis=0)  # Resample along the time axis
    return resampled_signal

# Example
resampled_signal = resample_signal(signal, original_rate=360, target_rate=500)
print("Resampled shape:", resampled_signal.shape)


Resampled shape: (902777, 2)


In [21]:
def normalize_signal(signal):
    """Normalize each lead (channel) independently."""
    normalized_signal = (signal - signal.mean(axis=0)) / signal.std(axis=0)
    return normalized_signal

# Example
normalized_signal = normalize_signal(resampled_signal)  # Apply to resampled signal
print("Normalized shape:", normalized_signal.shape)


Normalized shape: (902777, 2)


In [22]:
import numpy as np

In [23]:


def segment_signal(signal, window_size=1800, step_size=1800):
    """Segment the signal into overlapping/non-overlapping windows."""
    segments = []
    for start in range(0, signal.shape[0] - window_size + 1, step_size):
        segments.append(signal[start:start + window_size])
    return np.array(segments)

# Example
window_size = 1800  # 5 seconds at 360 Hz
segments = segment_signal(normalized_signal, window_size=window_size, step_size=window_size)
print("Segments shape:", segments.shape)  # Expected: (num_windows, 1800, 2)


Segments shape: (501, 1800, 2)


In [29]:
# Define normal and arrhythmia label mapping
normal_labels = {'N'}  # Normal beats
arrhythmia_labels = set(unique_labels) - normal_labels  # All other beats are arrhythmias

print("Arrhythmia labels:", arrhythmia_labels)  # Verify arrhythmia labels


Arrhythmia labels: {'A', 'V', '+'}


In [31]:
import numpy as np

def assign_labels_to_segments(segments, sample_indices, labels, window_size=1800):
    """
    Assigns a binary label (0: Normal, 1: Arrhythmia) to each segment based on annotations.
    """
    segment_labels = []

    for i in range(len(segments)):
        start = i * window_size  # Segment start time
        end = start + window_size  # Segment end time

        # ✅ Find indices where the sample annotations fall within this segment
        mask = (sample_indices >= start) & (sample_indices < end)
        segment_annotations = np.array(labels)[mask]  # Extract labels for this segment

        # ✅ Assign label: 1 if any arrhythmia is present, else 0 (normal)
        segment_label = 1 if any(label in arrhythmia_labels for label in segment_annotations) else 0
        segment_labels.append(segment_label)

    return np.array(segment_labels)

# ✅ Compute segment labels
segment_labels = assign_labels_to_segments(segments, np.array(sample_indices), np.array(labels), window_size=window_size)

print("Segment Labels:", segment_labels[:10])  # Check first 10 segment labels
print("Type of first label:", type(segment_labels[0]))


Segment Labels: [1 1 0 0 0 0 0 0 0 0]
Type of first label: <class 'numpy.int64'>


In [33]:
from imblearn.over_sampling import SMOTE
import numpy as np

# Convert to NumPy array
segment_labels_int = np.array(segment_labels, dtype=np.int64)

# ✅ Reshape segments to 2D (flattened representation)
num_samples, window_size, num_leads = segments.shape
segments_reshaped = segments.reshape(num_samples, -1)  # Shape: (num_samples, window_size * num_leads)

# ✅ Apply SMOTE only to the minority class (arrhythmia)
smote = SMOTE(sampling_strategy=0.5, random_state=42)  # Adjust as needed
segments_resampled, labels_resampled = smote.fit_resample(segments_reshaped, segment_labels_int)

# ✅ Reshape back to original shape (3D) for model training
segments_resampled = segments_resampled.reshape(-1, window_size, num_leads)  # Shape: (new_samples, window_size, num_leads)

# ✅ Print new class distribution
unique, counts = np.unique(labels_resampled, return_counts=True)
print("New balanced class distribution:", dict(zip(unique, counts)))


New balanced class distribution: {0: 467, 1: 233}


In [34]:
import numpy as np

unique, counts = np.unique(segment_labels, return_counts=True)
print("Class distribution:", dict(zip(unique, counts)))


Class distribution: {0: 467, 1: 34}


In [35]:
def load_mitdb_data(data_dir):
    # Load preprocessed data (signals and labels)
    signals = np.load(os.path.join(data_dir, "signals.npy"))
    labels = np.load(os.path.join(data_dir, "labels.npy"))
    return signals, labels


In [38]:
np.save("mitdb_signals.npy", segments_resampled)  # Save augmented ECG signals
np.save("mitdb_labels.npy", labels_resampled)    # Save corresponding labels


In [40]:
import os
import numpy as np

# Define file paths
signals_path = "./mitdb_signals.npy"
labels_path = "./mitdb_labels.npy"

# ✅ Check if files exist
if not os.path.exists(signals_path) or not os.path.exists(labels_path):
    print("🚨 Saving signals and labels before loading...")

    # Save signals and labels to disk
    np.save(signals_path, segments_resampled)  # Save augmented ECG signals
    np.save(labels_path, labels_resampled)     # Save corresponding labels
    print("✅ Files saved successfully!")

# ✅ Load the saved data
def load_mitdb_data(data_dir):
    """Load ECG signals and labels from .npy files."""
    signals = np.load(os.path.join(data_dir, "mitdb_signals.npy"))
    labels = np.load(os.path.join(data_dir, "mitdb_labels.npy"))
    return signals, labels

signals, labels = load_mitdb_data(".")  # Load signals and labels
print("✅ Loaded signals shape:", signals.shape)
print("✅ Loaded labels shape:", labels.shape)


✅ Loaded signals shape: (700, 1800, 2)
✅ Loaded labels shape: (700,)


In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class LocalLeadAttention(nn.Module):
    def __init__(self, input_size, num_classes, num_leads=2, hidden_size=256, num_heads=4, dropout_rate=0.3):
        """
        LocalLeadAttention model for multi-lead ECG classification.

        Args:
            input_size (int): Length of the input signal (e.g., 1800 time steps).
            num_classes (int): Number of output classes (e.g., 4).
            num_leads (int): Number of input channels/leads (e.g., 2).
            hidden_size (int): Size of the hidden layers.
            num_heads (int): Number of attention heads.
            dropout_rate (float): Dropout rate for regularization.
        """
        super(LocalLeadAttention, self).__init__()
        self.num_heads = num_heads
        self.hidden_size = hidden_size

        # Input layer for each lead
        self.input_layer = nn.Conv1d(
            in_channels=num_leads,
            out_channels=hidden_size,
            kernel_size=5,
            stride=1,
            padding=2
        )
        self.norm1 = nn.LayerNorm(hidden_size)
        self.dropout = nn.Dropout(dropout_rate)

        # Multi-head attention
        self.attention = nn.MultiheadAttention(
            embed_dim=hidden_size,
            num_heads=num_heads,
            batch_first=True
        )
        self.norm2 = nn.LayerNorm(hidden_size)

        # Fully connected layers
        self.fc1 = nn.Linear(hidden_size, 64)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        """
        Forward pass of the LocalLeadAttention model.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, time_steps, num_leads).

        Returns:
            torch.Tensor: Output logits of shape (batch_size, num_classes).
        """
        # Reshape input for Conv1d: (batch_size, num_leads, time_steps)
        x = x.permute(0, 2, 1)  # Shape: (batch_size, num_leads, time_steps)

        # Apply convolution to extract features
        x = self.input_layer(x)  # Shape: (batch_size, hidden_size, time_steps)
        x = self.dropout(self.norm1(x.permute(0, 2, 1)))  # Normalize and apply dropout
        x = x.permute(0, 2, 1)  # Shape: (batch_size, time_steps, hidden_size)

        # Apply multi-head attention
        attn_output, _ = self.attention(x, x, x)  # Self-attention
        x = self.norm2(attn_output + x)  # Add residual connection and normalize

        # Global average pooling over time
        x = x.mean(dim=1)  # Shape: (batch_size, hidden_size)

        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)  # Shape: (batch_size, num_classes)

        return x

# Define model parameters
input_size = 1800  # Signal length per segment
num_leads = 2      # Number of ECG leads (channels)
num_classes = 4    # Number of output classes ('N', 'A', '+', 'V')

# Initialize the model
model = LocalLeadAttention(
    input_size=input_size,
    num_leads=num_leads,
    num_classes=num_classes
)

# Print the model architecture
print(model)


LocalLeadAttention(
  (input_layer): Conv1d(2, 256, kernel_size=(5,), stride=(1,), padding=(2,))
  (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (attention): MultiheadAttention(
    (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
  )
  (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=256, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=4, bias=True)
)


In [43]:
import os
print(os.listdir("."))  # List files in the current directory


['.git', 'train_model.py', 'extract_leads_wfdb.py', 'mitdb_labels.npy', 'Dockerfile', 'dx_mapping_scored.csv', 'team_code.py', 'AUTHORS', 'weights.csv', 'model', 'helper_code.py', 'LICENSE', 'model_files', 'test_docker.sh', 'README.md', 'requirements.txt', 'evaluate_model.py', 'test_model.py', 'mitdb', 'dx_mapping_unscored.csv', 'mitdb_signals.npy', 'training.sh', 'model_code.py']


In [47]:
signals = np.load("mitdb_signals.npy")  # Load signals
labels = np.load("mitdb_labels.npy")    # Load labels


In [48]:
print("Signals shape:", signals.shape)  # Expected: (700, 1800, 2)
print("Labels shape:", labels.shape)    # Expected: (700,)


Signals shape: (700, 1800, 2)
Labels shape: (700,)


In [49]:
from sklearn.model_selection import train_test_split

# Define split ratios
train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

# Split into train and remaining (val + test)
signals_train, signals_temp, labels_train, labels_temp = train_test_split(
    signals, labels, test_size=(val_ratio + test_ratio), random_state=42, stratify=labels
)

# Split remaining into val and test
signals_val, signals_test, labels_val, labels_test = train_test_split(
    signals_temp, labels_temp, test_size=(test_ratio / (val_ratio + test_ratio)), random_state=42, stratify=labels_temp
)

# Print dataset sizes
print(f"Training set: {signals_train.shape}, Labels: {labels_train.shape}")
print(f"Validation set: {signals_val.shape}, Labels: {labels_val.shape}")
print(f"Test set: {signals_test.shape}, Labels: {labels_test.shape}")


Training set: (560, 1800, 2), Labels: (560,)
Validation set: (70, 1800, 2), Labels: (70,)
Test set: (70, 1800, 2), Labels: (70,)


In [50]:
!pip install imbalanced-learn


In [51]:
import numpy as np
from imblearn.over_sampling import SMOTE


In [52]:
# Check class distribution before SMOTE
unique, counts = np.unique(labels_train, return_counts=True)
print("Class distribution before SMOTE:", dict(zip(unique, counts)))



Class distribution before SMOTE: {0: 374, 1: 186}


In [54]:
from imblearn.over_sampling import SMOTE

# Flatten the data for SMOTE (SMOTE expects 2D input: (samples, features))
X_train_flat = signals_train.reshape(signals_train.shape[0], -1)  # (560, 1800*2)
y_train = labels_train

# Apply SMOTE with a better strategy (match class 0 count)
smote = SMOTE(sampling_strategy=1.0, random_state=42)  # Make both classes equal
X_resampled, y_resampled = smote.fit_resample(X_train_flat, y_train)

# Reshape back to original shape (samples, time_steps, leads)
signals_train_resampled = X_resampled.reshape(-1, 1800, 2)

# Print new class distribution
unique_resampled, counts_resampled = np.unique(y_resampled, return_counts=True)
print("Class distribution after SMOTE:", dict(zip(unique_resampled, counts_resampled)))

# Update labels_train with resampled labels
labels_train_resampled = y_resampled


Class distribution after SMOTE: {0: 374, 1: 374}


In [55]:
import torch
from torch.utils.data import TensorDataset, DataLoader

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(signals_train_resampled, dtype=torch.float32)
y_train_tensor = torch.tensor(labels_train_resampled, dtype=torch.long)

X_val_tensor = torch.tensor(signals_val, dtype=torch.float32)
y_val_tensor = torch.tensor(labels_val, dtype=torch.long)

X_test_tensor = torch.tensor(signals_test, dtype=torch.float32)
y_test_tensor = torch.tensor(labels_test, dtype=torch.long)

# Print shapes to verify
print("Train tensor shape:", X_train_tensor.shape, y_train_tensor.shape)
print("Val tensor shape:", X_val_tensor.shape, y_val_tensor.shape)
print("Test tensor shape:", X_test_tensor.shape, y_test_tensor.shape)


Train tensor shape: torch.Size([748, 1800, 2]) torch.Size([748])
Val tensor shape: torch.Size([70, 1800, 2]) torch.Size([70])
Test tensor shape: torch.Size([70, 1800, 2]) torch.Size([70])


In [56]:
# Create TensorDatasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)


In [57]:
# Define batch size
BATCH_SIZE = 32  # You can adjust this based on GPU memory

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Print number of batches
print("Train batches:", len(train_loader))
print("Validation batches:", len(val_loader))
print("Test batches:", len(test_loader))


Train batches: 24
Validation batches: 3
Test batches: 3


In [62]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ✅ Convert already defined tensors for training, validation, and testing
train_data, train_labels = X_train_tensor, y_train_tensor
val_data, val_labels = X_val_tensor, y_val_tensor
test_data, test_labels = X_test_tensor, y_test_tensor

# ✅ Create DataLoaders
batch_size = 32
train_loader = DataLoader(TensorDataset(train_data, train_labels), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(val_data, val_labels), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(test_data, test_labels), batch_size=batch_size, shuffle=False)

# ✅ Define the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LocalLeadAttention(input_size=256, num_classes=4).to(device)  # Updated input_size
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Function to downsample input from 1800 → 256
def downsample_input(inputs, target_size=256):
    return F.interpolate(inputs.permute(0, 2, 1), size=target_size, mode="linear").permute(0, 2, 1)

# ✅ Training & Validation Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device="cuda"):
    model.to(device)

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        all_preds, all_labels = [], []

        # 🚀 Training Loop
        for inputs, labels in train_loader:
            inputs, labels = downsample_input(inputs.to(device)), labels.to(device)  # Apply downsampling

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # ✅ Compute Training Metrics
        train_acc = accuracy_score(all_labels, all_preds)
        avg_train_loss = train_loss / len(train_loader)

        # 🚀 Validation Loop
        model.eval()
        val_loss = 0.0
        val_preds, val_labels_list = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = downsample_input(inputs.to(device)), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_labels_list.extend(labels.cpu().numpy())

        # ✅ Compute Validation Metrics
        val_acc = accuracy_score(val_labels_list, val_preds)
        avg_val_loss = val_loss / len(val_loader)
        precision = precision_score(val_labels_list, val_preds, average='macro', zero_division=0)
        recall = recall_score(val_labels_list, val_preds, average='macro', zero_division=0)
        f1 = f1_score(val_labels_list, val_preds, average='macro', zero_division=0)

        # ✅ Print Progress
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, "
              f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

    return model

# 🚀 Train the Model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device)

# ✅ Function to Evaluate Model on Test Set
def evaluate_model(model, test_loader, device="cuda"):
    model.eval()
    test_preds, test_labels_list = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = downsample_input(inputs.to(device)), labels.to(device)
            outputs = model(inputs)

            test_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            test_labels_list.extend(labels.cpu().numpy())

    # ✅ Compute Test Metrics
    test_acc = accuracy_score(test_labels_list, test_preds)
    precision = precision_score(test_labels_list, test_preds, average='macro', zero_division=0)
    recall = recall_score(test_labels_list, test_preds, average='macro', zero_division=0)
    f1 = f1_score(test_labels_list, test_preds, average='macro', zero_division=0)

    print(f"\n🚀 Test Set Evaluation:\n"
          f"Accuracy: {test_acc:.4f}, Precision: {precision:.4f}, "
          f"Recall: {recall:.4f}, F1-score: {f1:.4f}")

# ✅ Run Evaluation on Test Set
evaluate_model(trained_model, test_loader, device=device)


Epoch 1/20 - Train Loss: 0.9474, Train Acc: 0.4639 | Val Loss: 0.7583, Val Acc: 0.3286, Precision: 0.1643, Recall: 0.5000, F1-score: 0.2473
Epoch 2/20 - Train Loss: 0.5746, Train Acc: 0.7072 | Val Loss: 0.3155, Val Acc: 0.8571, Precision: 0.8381, Recall: 0.8381, F1-score: 0.8381
Epoch 3/20 - Train Loss: 0.3128, Train Acc: 0.8797 | Val Loss: 0.3566, Val Acc: 0.8286, Precision: 0.8198, Recall: 0.8612, F1-score: 0.8214
Epoch 4/20 - Train Loss: 0.2233, Train Acc: 0.9211 | Val Loss: 0.2754, Val Acc: 0.8714, Precision: 0.8542, Recall: 0.8932, F1-score: 0.8634
Epoch 5/20 - Train Loss: 0.2218, Train Acc: 0.9184 | Val Loss: 0.1791, Val Acc: 0.9000, Precision: 0.8841, Recall: 0.8922, F1-score: 0.8879
Epoch 6/20 - Train Loss: 0.1032, Train Acc: 0.9693 | Val Loss: 0.1660, Val Acc: 0.9429, Precision: 0.9289, Recall: 0.9463, F1-score: 0.9366


KeyboardInterrupt: 

In [67]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ✅ Convert already defined tensors for training, validation, and testing
train_data, train_labels = X_train_tensor, y_train_tensor
val_data, val_labels = X_val_tensor, y_val_tensor
test_data, test_labels = X_test_tensor, y_test_tensor

# ✅ Create DataLoaders
batch_size = 32
train_loader = DataLoader(TensorDataset(train_data, train_labels), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(val_data, val_labels), batch_size=batch_size, shuffle=False)
test_loader = DataLoader(TensorDataset(test_data, test_labels), batch_size=batch_size, shuffle=False)

# ✅ Define the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LocalLeadAttention(input_size=256, num_classes=4).to(device)  # Updated input_size
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# ✅ Function to downsample input from 1800 → 256
def downsample_input(inputs, target_size=256):
    return F.interpolate(inputs.permute(0, 2, 1), size=target_size, mode="linear").permute(0, 2, 1)

# ✅ Training & Validation Function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device="cuda"):
    model.to(device)
    best_val_acc = 0.0  # Track best validation accuracy

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        all_preds, all_labels = [], []

        # 🚀 Training Loop
        for inputs, labels in train_loader:
            inputs, labels = downsample_input(inputs.to(device)), labels.to(device)  # Apply downsampling

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

        # ✅ Compute Training Metrics
        train_acc = accuracy_score(all_labels, all_preds)
        avg_train_loss = train_loss / len(train_loader)

        # 🚀 Validation Loop
        model.eval()
        val_loss = 0.0
        val_preds, val_labels_list = [], []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = downsample_input(inputs.to(device)), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                val_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                val_labels_list.extend(labels.cpu().numpy())

        # ✅ Compute Validation Metrics
        val_acc = accuracy_score(val_labels_list, val_preds)
        avg_val_loss = val_loss / len(val_loader)
        precision = precision_score(val_labels_list, val_preds, average='macro', zero_division=0)
        recall = recall_score(val_labels_list, val_preds, average='macro', zero_division=0)
        f1 = f1_score(val_labels_list, val_preds, average='macro', zero_division=0)

        # ✅ Save best model based on validation accuracy
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), "best_model.pth")
            print("✅ Model saved!")

        # ✅ Print Progress
        print(f"Epoch {epoch+1}/{num_epochs} - "
              f"Train Loss: {avg_train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, "
              f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-score: {f1:.4f}")

    return model

# 🚀 Train the Model
trained_model = train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device=device)

# ✅ Load the Best Saved Model
trained_model.load_state_dict(torch.load("best_model.pth"))
trained_model.eval()

# ✅ Function to Evaluate Model on Test Set
def evaluate_model(model, test_loader, device="cuda"):
    model.eval()
    test_preds, test_labels_list = [], []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = downsample_input(inputs.to(device)), labels.to(device)
            outputs = model(inputs)

            test_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
            test_labels_list.extend(labels.cpu().numpy())

    # ✅ Compute Test Metrics
    test_acc = accuracy_score(test_labels_list, test_preds)
    precision = precision_score(test_labels_list, test_preds, average='macro', zero_division=0)
    recall = recall_score(test_labels_list, test_preds, average='macro', zero_division=0)
    f1 = f1_score(test_labels_list, test_preds, average='macro', zero_division=0)

    print(f"\n🚀 Test Set Evaluation:\n"
          f"Accuracy: {test_acc:.4f}, Precision: {precision:.4f}, "
          f"Recall: {recall:.4f}, F1-score: {f1:.4f}")

# ✅ Run Evaluation on Test Set
evaluate_model(trained_model, test_loader, device=device)


✅ Model saved!
Epoch 1/20 - Train Loss: 0.9649, Train Acc: 0.5067 | Val Loss: 0.5520, Val Acc: 0.7857, Precision: 0.7650, Recall: 0.7294, F1-score: 0.7413
✅ Model saved!
Epoch 2/20 - Train Loss: 0.5831, Train Acc: 0.6979 | Val Loss: 0.3613, Val Acc: 0.8429, Precision: 0.8207, Recall: 0.8275, F1-score: 0.8238
✅ Model saved!
Epoch 3/20 - Train Loss: 0.3760, Train Acc: 0.8436 | Val Loss: 0.2428, Val Acc: 0.9000, Precision: 0.8811, Recall: 0.9033, F1-score: 0.8901
Epoch 4/20 - Train Loss: 0.2237, Train Acc: 0.9184 | Val Loss: 0.2136, Val Acc: 0.8857, Precision: 0.8656, Recall: 0.8927, F1-score: 0.8756
✅ Model saved!
Epoch 5/20 - Train Loss: 0.1793, Train Acc: 0.9225 | Val Loss: 0.1387, Val Acc: 0.9714, Precision: 0.9676, Recall: 0.9676, F1-score: 0.9676
Epoch 6/20 - Train Loss: 0.1098, Train Acc: 0.9639 | Val Loss: 0.1787, Val Acc: 0.9143, Precision: 0.8958, Recall: 0.9251, F1-score: 0.9067
Epoch 7/20 - Train Loss: 0.0791, Train Acc: 0.9719 | Val Loss: 0.1792, Val Acc: 0.9143, Precision: 0

<ipython-input-67-0ad0d5d6e4d5>:97: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trained_model.load_state_dict(torch.load("best_model.pth"))



🚀 Test Set Evaluation:
Accuracy: 0.9571, Precision: 0.9570, Recall: 0.9475, F1-score: 0.9520
